In [2]:
import pandas as pd
import pickle

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_pickle(r"/Users/akshaykekuda/Desktop/CSR-SA/ScoringDetail_viw_subscore.p")
df = df.sort_values(by= ['RecordingDate', 'QGroupSequence', 'QuestionSequence']).copy()
cols = ['QGroupSequence', 'QGroupName','InteractionIdKey', 'QuestionSequence', 'QuestionText', 'QuestionType', 
        'QuestionPromptType', 'QuestionWeight', 'QuestionMin', 'QuestionMax', 'AnswerScore', 'RawAnswer', 'DisplayAnswer', 
        'UserComments']
calls_df = df[(df.QuestionnaireName == 'Call Interaction') & (df['RecordingDate'] >= pd.Timestamp(2021,1,1,0))].copy()
q_df = calls_df[cols]
temp = q_df[0:10]
temp = temp.reset_index(drop=True)
q_text =[]
for index, row in temp.iterrows():
    q_text.append(row['QuestionText'])

In [8]:
score_df = pd.DataFrame()#index = df['InteractionIdKey'].unique(), columns = q_text)
for row in calls_df.itertuples():
    score_df.loc[row.InteractionIdKey,row.QuestionText] = row.AnswerScore
    score_df.loc[row.InteractionIdKey,'WorkgroupQueue'] = row.WorkgroupQueue
    score_df.loc[row.InteractionIdKey,'CombinedPercentileScore'] = row.CombinedPercentileScore
#     grp = calls_df[calls_df.InteractionIdKey == idx].WorkgroupQueue.iloc[0]
score_df[q_text] = score_df[q_text].astype('int64')

In [9]:
score_df.sample(2)

,Greeting,WorkgroupQueue,CombinedPercentileScore,Professionalism,Confidence,Cross Selling,Retention,Creates Incentive,Product Knowledge,Education,Documentation,Processes
100129495250210331,3,Sales,58.97000000000000,5,2,0,4,1,5,0,3,0
200159040540210119,3,Sales,84.62000000000000,5,2,5,4,1,5,0,3,5


In [10]:
score_comment_df = pd.DataFrame()
for row in calls_df.itertuples():
    score_comment_df.loc[row.InteractionIdKey,row.QuestionText] = row.UserComments
    score_comment_df.loc[row.InteractionIdKey,'WorkgroupQueue'] = row.WorkgroupQueue
    score_comment_df.loc[row.InteractionIdKey,'CombinedPercentileScore'] = row.CombinedPercentileScore

In [11]:
import os
score_cols = score_df.columns.to_list()
dir = r"/Users/akshaykekuda/Desktop/CSR-SA/manual_score_transcriptions/output_mono_small"
df = pd.DataFrame(columns=(['csr_text', 'cust_text', 'csr_file_name', 'cust_file_name']) + score_cols )
for file in os.listdir(dir):
    if file.endswith('.sbv'):
        file_loc = dir + '/' + file
        f = open(file_loc, 'r')
        tscpt = f.read()
        f.close()
        arr = file.split("_")
        speaker = arr[0]
        id = arr[2].split('.')[0]
        if id in score_df.index:
            if speaker=='l':
                df.loc[id, ['cust_text', 'cust_file_name']] = [tscpt, file_loc]
            if speaker == 'r':
                df.loc[id, ['csr_text', 'csr_file_name']] = [tscpt, file_loc]
            df.loc[id, score_cols] = score_df.loc[id]                

In [12]:
df.sample(2)

csr_text  \
300109233330210317  00:00:00.000000,00:00:06.250000\nthank you for calling safe auto beaches manual tell me how can i help you today\n\n00:00:13.100000,00:00:18.060000\nok minimal can i have your policy number or the last four digits of your social security number\n\n00:00:26.550000,00:00:29.670000\nno what about um the last four digits of your social security number\n\n00:00:39.210000,00:00:42.520000\ni'm sorry i'm at um the language breaking up can you repeat that please\n\n00:00:48.340000,00:01:01.640000\nyes maam just what i'm know i don't know how to the free move a vehicle from the quotes and i just need to get access to the policy first so can have your policy number or last four digits of your social please\n\n00:01:06.910000,00:01:09.610000\nthank you and her data birth please\n\n00:01:13.900000,00:01:16.690000\nproduct thinking metal give me just a moment please\n\n00:01:22.980000,00:01:36.370000\ndo you met um can have her first on the phone number\n\n00:01:36.370000,00:01:42.170000\nonce you metal and to me i have the lecture to be speaking with\n\n00:01:45.470000,00:01:50.770000\nsame difference is well okay don't let me come from the rate to squeak reference\n\n00:01:50.770000,00:01:59.330000\nbefore we can make any changes the policy okay there right now middle eats two hundred and forty four dollars and if you caused that i have listed on the policy of met him\n\n00:01:59.330000,00:02:08.090000\nokay let them do you want me to act the ones to ask that vehicle first or your wants to make the burden in for\n\n00:02:19.080000,00:02:28.070000\nuh-huh okay metal when you to remove the tooth else in my forties scape no problem let me get the number before it can remove vehicle just a second\n\n00:02:31.440000,00:02:40.310000\nuh-huh okay well i could forgive skied doing your sports\n\n00:02:40.310000,00:02:49.530000\nto my mom just we look at a oh she got it\n\n00:02:49.530000,00:03:00.830000\nhello kim them or their rate was two hundred and forty four dollars and if we remove bag um the vehicle um the policy no rate i wont be two hundred and twenty dollars\n\n00:03:04.960000,00:03:08.950000\nyeah okay so did you want me to say the endorsements\n\n00:03:13.820000,00:03:17.200000\ni get\n\n00:03:17.200000,00:03:30.460000\noh yes i yes maam before i came to you he said if it changes from the policy by removing vehicle did you need to make any other changes solve the policy or just these a sticky out the vehicle\n\n00:03:34.390000,00:03:40.810000\nokay no problem\n\n00:03:46.560000,00:03:55.880000\nno problem no them uh she didn't remove the vehicle and actually we can both rate will go down okay so yes women to remove the vehicle parents\n\n00:03:55.880000,00:04:11.900000\ncan make the payment bam uh one thing i was almost in madam because certainly changed because i need to remove vehicle from the policy and you can mail gym you copy of your certificate of insurance bickering she'll page and cards so i am the email address there on file eve she tends to\n\n00:04:11.900000,00:04:20.540000\nl brown an twenty two at yahoo dot com is that correct\n\n00:04:20.540000,00:04:28.300000\nthank you metal okay which is probably let me stayed the change of solve the policy on them to email you and your copy up your proof of insurance and i'll be right back with\n\n00:04:34.950000,00:04:41.080000\noh just if any to email you proof of insurance to save the change of sound the policy okay\n\n00:04:41.080000,00:04:46.920000\npeople drop three calls i mean just bounce ill give me just a moment okay i'll be right there\n\n00:04:51.640000,00:04:58.090000\nokay me says brown think you very much for your patients okay but i'm just so sad quaint recap which is\n\n00:04:58.090000,00:05:04.410000\nyour for a from from the the policy their rate now will be two hundred and twenty dollars a month\n\n00:05:04.430000,00:05:12.090000\num also i just emailed to a copy of your day for each each certificate of insurance and the i_d cards a

In [10]:
csr_df = df[~(df.csr_text.isna())]

In [11]:
from nltk import word_tokenize

In [12]:
import os
java_path = r"C:\Program Files\Java\jre1.8.0_291\bin\java.exe"
os.environ['JAVAHOME'] = java_path

In [13]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize, sent_tokenize,WordPunctTokenizer
st = StanfordNERTagger(r'C:\Users\kekuak\Documents\stanford-ner\stanford-corenlp-4.2.1-models-english\edu\stanford\nlp\models\ner\english.all.3class.caseless.distsim.crf.ser.gz',
					   r'C:\Users\kekuak\Documents\stanford-ner\stanford-ner.jar',
					   encoding='utf-8')

In [14]:
def check_name(sent):
    org_flag = False; name_flag = False
#     porter = PorterStemmer()
    print(sent)
    words = WordPunctTokenizer().tokenize(sent)
#     words = [porter.stem(word) for word in words]
    tagged = st.tag(words)
    for i, ner in enumerate(tagged):
        if words[i] in ['safe', 'auto', 'safeauto']:
            org_flag = True
        elif words[i] == 'name':
            name_flag = True
        if ner[1] == 'PERSON':
            name_flag = True
            print("Loc: {} | Person is: {}".format(i, ner[0]))
        if ner[1] == 'LOCATION':
            print("Loc: {} | Location is: {}".format(i, ner[0]))
        if ner[1] == 'ORGANIZATION':
            print("Loc: {} | Organization is: {}".format(i, ner[0]))
    return org_flag, name_flag

In [15]:
def predict_greeting(row):
    id = row.Index
    greeting_df.loc[id,'Greeting Score' ] = row.Greeting
        
    greeting_df.loc[id,'User Comments' ] = score_comment_df.loc[id, 'Greeting']
    i = 1
    text = row.csr_text
    greet = text.split('\n')
    opening_line = greet[i]
    while len(word_tokenize(opening_line)) < 4:
        opening_line = greet[3*i+1]
        i = i+ 1
    open_time = greet[3*(i-1)].split(',')[0]
    
    greeting_df.loc[id,'Open Time' ] = open_time
    greeting_df.loc[id,'Open Sentence' ] = opening_line
    
    org_flag, name_flag = check_name(opening_line)
    greeting_df.loc[id,'org_flag' ] = org_flag
    greeting_df.loc[id,'name_flag' ] = name_flag
    
    date_time_obj = datetime.datetime.strptime(open_time, '%H:%M:%S.%f')
    if (date_time_obj.second > 5):
        greeting_df.loc[id,'Time Score' ] = True
    else:
        greeting_df.loc[id,'Time Score' ] = False    

In [120]:
def predict_confidence(row):
    id = row.Index
    confidence_df.loc[id,'Confidence Score' ] = row.Confidence
    confidence_df.loc[id,'Prof Comments' ] = score_comment_df.loc[id, 'Professionalism']    
    confidence_df.loc[id,'Conf Comments' ] = score_comment_df.loc[id, 'Confidence']
    confidence_df.loc[id,'Silence'] = False
    sil_arr = []
    text = row.csr_text
    sent_tok = text.split('\n')
    time = sent_tok[0]
    end_time = time.split(',')[1]
    end_time_obj = datetime.datetime.strptime(end_time, '%H:%M:%S.%f')
    for i in range(3, len(sent_tok) -1, 3):
        time = sent_tok[i]
        start_time = time.split(',')[0]
        start_time_obj = datetime.datetime.strptime(start_time, '%H:%M:%S.%f')
        delta = start_time_obj - end_time_obj
        if delta.total_seconds()>20:
            confidence_df.loc[id,'Silence'] = True
            sil_arr.append((end_time, start_time))
        end_time = time.split(',')[1]
        end_time_obj = datetime.datetime.strptime(end_time, '%H:%M:%S.%f')     
    confidence_df.loc[id,'Silence Gaps'] = len(sil_arr)

In [92]:
def predict_professionalism(row):
    id = row.Index
    professionalism_df.loc[id,'Prof Score' ] = row.Professionalism
    professionalism_df.loc[id,'Prof Comments' ] = score_comment_df.loc[id, 'Professionalism']    
    if (isinstance(row.csr_text, str) and  isinstance(row.cust_text, str)):
        try:         
            count = 0
            csr_text = row.csr_text
            csr_sent_tok = csr_text.split('\n')
            csr_time = [csr_sent_tok[i] for i in range(0, len(csr_sent_tok) -1, 3)]
            cust_text = row.cust_text
            cust_sent_tok = cust_text.split('\n')
            cust_time = [cust_sent_tok[i] for i in range(0, len(cust_sent_tok) -1, 3)]
            for i in range(len(cust_time)):
                cust_start = cust_time[i].split(',')[0]
                cust_start_obj = datetime.datetime.strptime(cust_start, '%H:%M:%S.%f')
                cust_end = cust_time[i].split(',')[1]
                cust_end_obj = datetime.datetime.strptime(cust_end, '%H:%M:%S.%f')
                for j in range(len(csr_time)):
                    csr_start = csr_time[j].split(',')[0]
                    csr_end = csr_time[j].split(',')[1]
                    csr_start_time_obj = datetime.datetime.strptime(csr_start, '%H:%M:%S.%f')
                    if csr_start_time_obj >= cust_start_obj and csr_start_time_obj <=cust_end_obj:
                        count+=1
            professionalism_df.loc[id,'Interruption Count' ] = count
            
        except:
            print(csr_text)
                

In [121]:
import datetime
greeting_df = pd.DataFrame(columns = ['Greeting Score', 'Predicted Score', 'User Comments', 'Open Time', 'Open Sentence', 'Time Score',
                                      'Name Score'])
confidence_df = pd.DataFrame(columns = ['Confidence Score', 'Predicted Score', 'Prof Comments', 'Conf Comments','Silence', 'Silence Gaps'])
professionalism_df = pd.DataFrame(columns = ['Professionalism Score', 'Predicted Score', 'Prof Comments', 'Conf Comments','Interruption Count'])

for row in csr_df.itertuples():
    predict_professionalism(row)
#     predict_greeting(row)
    predict_confidence(row)


In [65]:
temp = csr_df[~(csr_df['cust_text'].isna() | csr_df['csr_text'].isna())]

In [94]:
professionalism_df[professionalism_df['Interruption Count'] <10]

,Professionalism Score,Predicted Score,Prof Comments,Conf Comments,Interruption Count,Prof Score
300120313540210129,NaN,NaN,None,NaN,7,5.0
300120648650210505,NaN,NaN,None,NaN,0,5.0
300122636330210319,NaN,NaN,agent was polite however there were moments of dead air.,NaN,3,0.0
300123629940210129,NaN,NaN,We were polite and professional throughout the call,NaN,8,5.0
300130447530210320,NaN,NaN,None,NaN,4,0.0
300130474030210320,NaN,NaN,We were polite and professional throughout the call,NaN,7,5.0
300135509930210321,NaN,NaN,"CSR had a good tone and manner, she was willing and polite.",NaN,5,5.0
300135515340210201,NaN,NaN,"CSR had a great tone and maner, he was willing and polite.",NaN,6,5.0
300135804640210201,NaN,NaN,GREAT JOB BY OFFERING IMEDIATE ASSISTANCE,NaN,7,5.0
300138688640210201,NaN,NaN,"CSR had a good tone and manner, he was willing and polite.",NaN,9,5.0


In [122]:
dead_arr = []
for row in confidence_df['Prof Comments'].items():
    if row[1] is None:
        continue
    comment = WordPunctTokenizer().tokenize(row[1])
    if ('dead' in comment) or ('silence' in comment):
        print(row[0], row[1])
        dead_arr.append(row[0])
for row in confidence_df['Conf Comments'].items():
    if row[1] is None:
        continue
    comment = WordPunctTokenizer().tokenize(row[1])
    if ('dead' in comment) or ('silence' in comment):
        print(row[0], row[1])
        dead_arr.append(row[0])

300122636330210319 agent was polite however there were moments of dead air.
200131644550210419 let´s work on dead air under 30 secs
200146960280210308 Try and reduce longer dead silences, one long silence was over a minute in length
200183013030210313 Long silence gap
200185761730210313 long silence gap
200157700240210119 We were very clear with the question ask, well explain, very confident.
However we had several caps of dead air and customer ask for us if we were still on the line.
300119282950210505 There was a lapse of dead air for over 20 seconds.
300122443140210129 Dead air from min4:24-7:47
Know on how I see the recording u might wanted to use Hold but it wasen´t use properly, we can see the dead air in between the yellow triagle that indicates when we use the dead air and both of them are for 1sec each.
300122691250210505 There was a laps of dead air for over 20 seconds.
300122694830210319 several gapses of dead air ove 2 min
300135788930210322 Threre was a lapse of dead air f

In [123]:
confidence_df.loc[dead_arr, 'Silence Score'] = 1

In [124]:
confidence_df['Silence Score'] = (~confidence_df['Silence Score'].isna())*1

In [125]:
confidence_df['Silence'] = confidence_df['Silence'] *1

In [128]:
confidence_df[confidence_df.Silence == 0]

,Confidence Score,Predicted Score,Prof Comments,Conf Comments,Silence,Silence Gaps,Silence Score
100161465830210103,2,NaN,We were polite and professional throughout the call,We took ownership of the call and had a clear and confident tone,0,0,0
200158561640210119,2,NaN,"CSR had a good tone and manner, he was willing and polite.",CSR sounded confident and sure of his statement.,0,0,0
200164881340210120,2,NaN,"CSR had a great tone and maner, he was willing and polite.",CSR sounded confident and sure of his statement.,0,0,0
200166004940210120,2,NaN,"CSR had a good tone and manner, she was willing and polite.",CSR sounded confident and sure of her statement.,0,0,0
200166037240210120,2,NaN,agent was polite and engaging throughout interaction,agent took ownership of the call and was clear with responses.,0,0,0
...,...,...,...,...,...,...,...
300167846440210206,2,NaN,"CSR had a great tone and maner, he was willing and polite.",CSR sounded confident and sure of his statement.,0,0,0
300168845540210206,2,NaN,"Good job on being professional, courteous, polite, engaging and saying please and thank you.",Good job on sounding confident during the call and being clear on the call.,0,0,0
300184650440210209,2,NaN,"CSR had a good tone and manner, he was willing and polite.",CSR sounded confident and sure of his statement.,0,0,0
300193949940210211,2,NaN,We did have a good active listening when talking to customer and sounded nicely with customer.,We mentioned information with a good tone and sounded confidently.,0,0,0


In [145]:
len(temp)

131

In [148]:
temp = score_comment_df[score_df['Professionalism'] == 0]
dead_arr = []
for row in temp['Professionalism'].items():
    if row[1] is None:
        dead_arr.append(row[0])
        continue
    comment = WordPunctTokenizer().tokenize(row[1])
    if ('dead' in comment) or ('silence' in comment):
        print(row[0], row[1])
    else:
        dead_arr.append(row[0])

100161076730210103 This is the 3rd time we advise to work with greeting and call was silence for 2 min and 10 seconds.
200105179640210109 long silence gap no refreshinig customer and no active liistening
100119687750210216 silence gaps
100135980550210218 Juan needs to work on silence gap and verification, customer offered policy numnber 2 times
100140214550210219 Diego needs to work on hold time, silence gap.
100156146860210222 silence gap and active listening
100199873260210301 A lot of silence gap and feedback will be provided
200163857730210310 agent was polite however there were moments of dead air.
200183013030210313 Long silence gap
200185761730210313 long silence gap
300122636330210319 agent was polite however there were moments of dead air.
300136845030210322 silence gaps and warm transfer
300175311840210327 Long silence gap and tone of voice
300176292240210327 Long silence gap and customer called 2 times and we didnt not answer. also ask if we still safeauto and we did not pro

In [149]:
score_comment_df.loc[dead_arr]

,Greeting,CombinedPercentileScore,Professionalism,Confidence,Cross Selling,Retention,Creates Incentive,Product Knowledge,Education,Documentation,Processes
100155182230210101,We answered the call in a timely manner and properly introduced ourselves,87.18000000000000,We were rude and unprofessional by not showing any kind of empathy for the customers issue,We took ownership of the call and had a clear and confident tone,There was no area of opportunity to cross sell,Rep identified the customers need and provided the proper solution,We incentivized the call by providing good customer service,We used all available resources to provide assistance and we demonstrated good produck knowledge,We provided the proper information based on the customers need,Accurate and properdocumentation,Rep followed the proper procedure throughout the call
100158755430210102,None,71.79000000000000,took almost 20 sec to answer call,None,None,None,None,None,didnt provided right information from the begining,None,None
100158932930210102,None,87.18000000000000,Place customer on hold for 5 minutes and we have to refresh customer every 2 minutes,None,None,None,None,None,None,None,None
100159138530210102,None,35.90000000000000,None,None,None,None,None,Francisco asked csd rep to explaine customer they can pay rental if they have in policy. out the pocket / we are not to provided that information and later on he explained he is not adjuster.,None,None,None
100159160130210102,None,84.62000000000000,Need to work on tone of voice,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
200163846150210425,We answered the call on time and properly introduced ourselves,58.97000000000000,"Please lets work on letting the customer answer the questions we ask him without interrupting and talking over him/her, We did it multiple times in this call when asking the underwriting questions",After the customer provided the owners name of the vehicle we had dead air of alost 2 minutes that interrupted the call flow.,We missed the area of opportunity to offer additional services such recreational vehicle insurance and ForeverCar insurance.,We identified the customers need and provided the proper solution,Rep created an incentive by providing good customer service throughout the call.,We assisted the customers need using all available resources and demonstrated good product knowledge,New Monthly rate was not provided,Accurate and proper documentation,Rep followed the proper procedure throughout the call
200177443450210427,proper greeting used,87.18000000000000,TL was couteous and professional as the call went along. TL did start off convo with an aggressive tone and this caused the call to start poorly>,"TL took ownership of the situation, responses were clear.",no opportunity,None,TL explained about the rate increase with the address change and was able to de-escalate the situation.,None,TL educated about the rate increase due to address change and was able to explain the reason why address factors in. TL also advised monthly rate and what next bill will be.,notes were regarding the interaction and what was discussed.,None
300104199150210503,None,87.18000000000000,Jorge needs to let customer speak out first,None,None,None,None,None,Jorge was able to provide information needed for customer,Jorge did place notes,Jorge followed all the process
300111989950210504,None,71.79000000000000,None,None,None,None,None,None,None,None,None


In [153]:
with open('ids_att1.txt', 'w') as f:
    for item in dead_arr:
        f.write("%s\n" % item)

In [126]:
from sklearn.metrics import classification_report
print(classification_report(confidence_df['Silence Score'].to_list(), confidence_df['Silence'].to_list(), 
                            target_names = ['Silence Absent', 'Silence Present']))

                 precision    recall  f1-score   support

 Silence Absent       0.99      0.48      0.65       773
Silence Present       0.10      0.92      0.19        50

       accuracy                           0.51       823
      macro avg       0.55      0.70      0.42       823
   weighted avg       0.94      0.51      0.62       823



In [105]:
greeting_df_org = greeting_df.copy()

In [106]:
greeting_df_org

,Greeting Score,User Comments,Open Time,Open Sentence,Time Score,Name Score,Predicted Score,org_flag,name_flag
100161465830210103,0,We took an excessive amount of time to greet the customer,00:00:26.350000,hello thank you for calling safe auto my names crystal how can i help you,True,True,0,True,False
200157700240210119,3,"Very good job with intro, we informed called will be recorded and the time it will take to file the claim and customer understood.",00:00:00.090000,main calling safeauto this angry have the nicest you and you are are new record am i,False,True,0,False,False
200158561640210119,3,CSR identified himself and greeting was prompt.,00:00:00.000000,good morning thank the safe auto this is about how can i help his leg,False,True,0,True,False
200161163540210119,3,She answered promptly and stated name,00:00:02.200000,thank you for calling safe auto my name is paddy how can i help you today,False,False,3,True,True
200163675540210119,3,We answered the call in a timely manner and properly introduced ourselves,00:00:03.340000,thank you for calling safe auto my name is joe how many uh fifth you,False,False,3,True,True
...,...,...,...,...,...,...,...,...,...
300196498740210211,3,None,00:00:00.000000,thank you for calling safe auto this is angel how can a hobby,False,True,0,True,False
300196543140210211,3,we identify ourselves and mention the name of the company.,00:00:00.000000,safe auto my names i hope you,False,True,0,True,False
300196960440210211,3,CSR identified himself and greeting was prompt.,00:00:02.960000,hello thank you for calling safe auto be how can i help you,False,True,0,True,False
300197062340210211,3,GREAT JOB BY OPENING THE CALL ON TIME,00:00:00.000000,thank you for calling safe auto my name is kim how may i assist you,False,False,3,True,True


In [29]:
greeting_df[(greeting_df['org_flag'])]

,Greeting Score,User Comments,Open Time,Open Sentence,Time Score,Name Score,Predicted Score,org_flag,name_flag
100161465830210103,0,We took an excessive amount of time to greet t...,00:00:26.350000,hello thank you for calling safe auto my names...,True,NaN,NaN,True,False
200158561640210119,3,CSR identified himself and greeting was prompt.,00:00:00.000000,good morning thank the safe auto this is about...,False,NaN,NaN,True,False
200161163540210119,3,She answered promptly and stated name,00:00:02.200000,thank you for calling safe auto my name is pad...,False,NaN,NaN,True,True
200163675540210119,3,We answered the call in a timely manner and pr...,00:00:03.340000,thank you for calling safe auto my name is joe...,False,NaN,NaN,True,True
200164881340210120,3,CSR identified himself and greeting was prompt.,00:00:00.110000,hello ticket for calling safe auto my name is ...,False,NaN,NaN,True,True
...,...,...,...,...,...,...,...,...,...
300193949940210211,3,"We did answer on timely manner, Good Job!",00:00:00.000000,thank you for calling safe auto group agency m...,False,NaN,NaN,True,True
300196498740210211,3,None,00:00:00.000000,thank you for calling safe auto this is angel ...,False,NaN,NaN,True,False
300196543140210211,3,we identify ourselves and mention the name of ...,00:00:00.000000,safe auto my names i hope you,False,NaN,NaN,True,False
300196960440210211,3,CSR identified himself and greeting was prompt.,00:00:02.960000,hello thank you for calling safe auto be how c...,False,NaN,NaN,True,False


In [38]:
greeting_df[(greeting_df['org_flag'] == False)]

,Greeting Score,User Comments,Open Time,Open Sentence,Time Score,Name Score,Predicted Score,org_flag,name_flag
200157700240210119,3,"Very good job with intro, we informed called w...",00:00:00.090000,main calling safeauto this angry have the nice...,False,NaN,NaN,False,False
300119265130210318,3,we identify ourselves and mention the name of ...,00:00:03.120000,thank you for contacting safeauto this is matt...,False,NaN,NaN,False,True
300119311430210318,3,"We did answer on timely manner, Good Job!",00:00:00.000000,and kelly how can help you this is rod spooky,False,NaN,NaN,False,True
300120121040210129,0,Make sure to promptly greet caller when calls ...,00:00:09.560000,think it's going to miss i miss him say how ca...,True,NaN,NaN,False,False
300120313540210129,0,Try to greet calls within 3-5 seconds,00:00:09.220000,think it's cutting them aside my names say how...,True,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...
300163529640210325,3,None,00:00:00.000000,thinking of a z total let me ask you is due,False,NaN,NaN,False,False
300167562040210206,3,CSR identified herself and greeting was prompt.,00:00:00.110000,thank you so much for calling safeauto this is...,False,NaN,NaN,False,True
300167676040210206,3,CSR idenitifed herself and greeting was prompt.,00:00:00.100000,thank you so much for calling safeauto this is...,False,NaN,NaN,False,False
300168506340210326,0,"we answer at sec12, and we seem not to be read...",00:00:18.340000,hello this is so yeah i'm doing great how are ...,True,NaN,NaN,False,False


In [53]:
greeting_df['Name Score'] = ~(greeting_df.org_flag & greeting_df.name_flag)

In [55]:
greeting_df['Predicted Score'] = (~(greeting_df['Time Score'] | greeting_df['Name Score'])*3)

In [91]:
greeting_df[greeting_df['Name Score']].sample(5)

,Greeting Score,User Comments,Open Time,Open Sentence,Time Score,Name Score,Predicted Score,org_flag,name_flag
200146155250210421,3,Prompt greeting and identified self,00:00:00.000000,hi cameras safe auto can help you,False,True,0,True,False
300107461440210127,3,CSR identified herself and greeting was prompt.,00:00:02.940000,i think its neck for calling safe auto mean,False,True,0,True,False
300196498740210211,3,None,00:00:00.000000,thank you for calling safe auto this is angel how can a hobby,False,True,0,True,False
200171084050210426,3,None,00:00:01.420000,thank you for calling safe auto this is jessica how can i help you,False,True,0,True,False
300155580640210204,3,Prompt greeting and identified self,00:00:00.000000,um what do you,False,True,0,False,False


In [90]:
greeting_df[~greeting_df['Name Score']].sample(5)

,Greeting Score,User Comments,Open Time,Open Sentence,Time Score,Name Score,Predicted Score,org_flag,name_flag
300107077140210127,3,CSR identified himself and greeting was prompt-,00:00:00.100000,thank you for calling safe auto my name is jay how can i assist you,False,False,3,True,True
200148681340210116,3,CSR identified himself and greeting was prompt.,00:00:03.110000,thank you for calling safe auto my names jose how can help you today,False,False,3,True,True
300153655640210204,3,CSR identified himself and greeting was prompt.,00:00:00.070000,things are going to have auto my name is oscar how may i help,False,False,3,True,True
300134960440210201,3,Good job on answering call on time and introducing ourselves and be willing to help.,00:00:00.070000,thank you for calling safe auto this is barbara how can i help you,False,False,3,True,True
200188809040210123,3,CSR identified herself and greeting was prompt.,00:00:02.540000,thank you for calling safe auto this is jackie tony how can i help you today,False,False,3,True,True


In [101]:
TN = greeting_df[(greeting_df['Greeting Score'] == 0) & (greeting_df['Predicted Score'] == 0)]
PN = greeting_df[(greeting_df['Predicted Score'] == 0)]

In [103]:
PN

,Greeting Score,User Comments,Open Time,Open Sentence,Time Score,Name Score,Predicted Score,org_flag,name_flag
100161465830210103,0,We took an excessive amount of time to greet the customer,00:00:26.350000,hello thank you for calling safe auto my names crystal how can i help you,True,True,0,True,False
200157700240210119,3,"Very good job with intro, we informed called will be recorded and the time it will take to file the claim and customer understood.",00:00:00.090000,main calling safeauto this angry have the nicest you and you are are new record am i,False,True,0,False,False
200158561640210119,3,CSR identified himself and greeting was prompt.,00:00:00.000000,good morning thank the safe auto this is about how can i help his leg,False,True,0,True,False
300116342730210318,3,we identify ourselves and mention the name of the company.,00:00:02.560000,hello thank you for calling safe auto this should is fifteen that country,False,True,0,True,False
300119194050210505,3,None,00:00:02.600000,thank you for calling safe auto my names either how can help you today,False,True,0,True,False
...,...,...,...,...,...,...,...,...,...
300184658240210209,3,We answered the call on time and properly introduced ourselves,00:00:00.000000,thank you for calling safe auto monday and say how can they help you,False,True,0,True,False
300196498740210211,3,None,00:00:00.000000,thank you for calling safe auto this is angel how can a hobby,False,True,0,True,False
300196543140210211,3,we identify ourselves and mention the name of the company.,00:00:00.000000,safe auto my names i hope you,False,True,0,True,False
300196960440210211,3,CSR identified himself and greeting was prompt.,00:00:02.960000,hello thank you for calling safe auto be how can i help you,False,True,0,True,False


In [85]:
from sklearn.metrics import classification_report
print(classification_report(greeting_df['Greeting Score'].to_list(), greeting_df['Predicted Score'].to_list()))

              precision    recall  f1-score   support

           0       0.07      0.96      0.12        27
           3       1.00      0.53      0.69       796

    accuracy                           0.54       823
   macro avg       0.53      0.75      0.41       823
weighted avg       0.97      0.54      0.67       823



In [ ]:
f = open(r'C:\Users\kekuak\Documents\manual_score_transcriptions\transcripts\dual_ch_trans\IRCall_100100429850210212.txt', 'r')
tscpt = f.read()
f.close()
sent = sent_tokenize(tscpt)
check_name(sent[0])

In [92]:
check_name('thank you for calling safe auto this is jessica how can i help you')

thank you for calling safe auto this is jessica how can i help you


(True, False)

In [93]:
check_name('thank you for calling safe auto this is barbara how can i help you')

thank you for calling safe auto this is barbara how can i help you
Loc: 8 | Person is: barbara


(True, True)

In [ ]:
greeting_df['Open Sentence'][0]

In [64]:
greeting_df.loc['200158081340210119']

Greeting Score                                                     3
User Comments        CSR identified himself and greeting was prompt.
Open Time                                            00:00:02.290000
Open Sentence      hi vicky much for calling safe auto my name is...
Time Score                                                     False
Name Score                                                     False
Predicted Score                                                    3
org_flag                                                        True
name_flag                                                       True
Name: 200158081340210119, dtype: object

In [95]:
greeting_df.to_pickle('greeting_df.p')

In [96]:
csr_df

csr_text  \
100161465830210103                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [26]:
df

csr_text  \
100104185650210212                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [45]:
score_comment_df.loc[df[~df['csr_text'].isna()].index, ['Cross Selling', 'WorkgroupQueue']].to_excel('small_cross_selling.xlsx')

In [46]:
temp =df[~df['csr_text'].isna()]

In [49]:
temp[temp['Cross Selling']==0]

csr_text  \
100114948650210215                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [24]:
score_comment_df[['Cross Selling', 'WorkgroupQueue']]

,Cross Selling,WorkgroupQueue
100155182230210101,There was no area of opportunity to cross sell,Customer Service
100155356030210101,She asked about rental and recreational,Sales
100155481230210101,Elizabeth did good job on offering HO insurance and also ask if customer have any recreational vehicles.,Sales
100156064130210101,"Crystal did good job on offering renters insurance, and also ask if customer have any recreational vehicles.",Sales
100156109530210101,WE DID NOT ASKED FOR RENTERS HOMEOWNERS AND FOR RECREATIONAL VEHICLES,Sales
...,...,...
300133490550210507,She did offer Forever Car,Customer Service
300133595050210507,She offered all cross selling products,Sales
300133775350210507,She didn't have to offer Forever Car due to vehicle didn't qualify.,Customer Service
300134565650210507,She did cross sell the policy,Sales
